In [ ]:
import pandas as pd
import requests
import cv2
import mediapipe as mp
import os

In [ ]:
# Read CSV as a DataFrame
csv_file_path = 'tagesschau_data.csv'
tagesschau_links_df = pd.read_csv(csv_file_path)
tagesschau_links_df.head()

In [ ]:
video_date = tagesschau_links_df['date'].iloc[1]
video_url = tagesschau_links_df['360p'].iloc[1]
subtitle_url = tagesschau_links_df['subtitles'].iloc[1]

video_date, video_url, subtitle_url

In [ ]:
# Function to download a file from a URL
def download_file(url, file_name):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded {file_name}")
    else:
        print(f"Failed to download {file_name}. Status code: {response.status_code}")

# Download the video file
video_file_name = "video_" + video_date + ".mp4"
download_file(video_url, video_file_name)

# Download the subtitle file
subtitle_file_name = "subtitle_" + video_date + ".xml"
download_file(subtitle_url, subtitle_file_name)

In [ ]:
# Initialize Mediapipe Holistic and Drawing utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Start capturing video
cap = cv2.VideoCapture(video_file_name)

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

# Define the frame range to process
start_frame = int(input("Enter the start frame: "))
end_frame = int(input("Enter the end frame: "))

frame_count = 0
landmarks_data = []

output_folder = os.path.join(os.getcwd(), video_date)

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2,
                          enable_segmentation=True,
                          refine_face_landmarks=True) as holistic:
    # Modified part of the loop
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Skip frames outside the selected range
        if frame_count < start_frame or frame_count > end_frame:
            continue

        # Get frame dimensions
        height, width, _ = frame.shape

        # Crop the right half of the frame
        right_half = frame[:, width // 2:]

        # Convert the cropped frame to RGB for Mediapipe processing
        image = cv2.cvtColor(right_half, cv2.COLOR_BGR2RGB)

        # Process the cropped frame to get landmarks
        results = holistic.process(image)

        # Convert the processed frame back to BGR for visualization
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Collect landmarks data
        if results.pose_landmarks:
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                landmarks_data.append({
                    'frame': frame_count,
                    'landmark_type': 'pose',
                    'landmark_index': idx,
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z,
                    'visibility': landmark.visibility
                })
        if results.face_landmarks:
            for idx, landmark in enumerate(results.face_landmarks.landmark):
                landmarks_data.append({
                    'frame': frame_count,
                    'landmark_type': 'face',
                    'landmark_index': idx,
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z
                })
        if results.left_hand_landmarks:
            for idx, landmark in enumerate(results.left_hand_landmarks.landmark):
                landmarks_data.append({
                    'frame': frame_count,
                    'landmark_type': 'left_hand',
                    'landmark_index': idx,
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z
                })
        if results.right_hand_landmarks:
            for idx, landmark in enumerate(results.right_hand_landmarks.landmark):
                landmarks_data.append({
                    'frame': frame_count,
                    'landmark_type': 'right_hand',
                    'landmark_index': idx,
                    'x': landmark.x,
                    'y': landmark.y,
                    'z': landmark.z
                })

        # Save the processed frame as an image every X frames
        if frame_count % 1 == 0:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            if results.face_landmarks:
                mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            # Save the image to the output folder
            output_file = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(output_file, image)
            print(f"Saved frame {frame_count} to {output_file}")

        # Exit on pressing 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release video capture and close resources
    cap.release()
    cv2.destroyAllWindows()

# Convert landmarks data to DataFrame
landmarks_df = pd.DataFrame(landmarks_data)

In [ ]:
landmarks_df.to_csv('landmarks_' + video_date + '.csv', index=False)

In [ ]:
landmarks_df.to_parquet('landmarks_' + video_date + '.parquet', engine='pyarrow')